<a href="https://colab.research.google.com/github/Gio-Choi/Stock_Regime_Analysis/blob/main/MLP_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Capstone_Dataset.csv')

In [3]:
df

,Unnamed: 0,^GSPC,VIXCLS,CPIAUCSL,M2SL,FEDFUNDS,T10Y2Y,AAA,UNRATE,PCUOMFGOMFG,USPHCI,IQ,S&P_pct
0,2004-03-01,1155.969971,14.44,187.100,6150.1,1.00,2.33,5.33,5.8,140.3,90.98,103.0,1.0
1,2004-04-01,1132.170044,16.65,187.400,6191.2,1.00,2.26,5.73,5.6,141.8,91.28,103.7,0.0
2,2004-05-03,1117.489990,16.62,188.200,6268.1,1.00,2.19,6.04,5.6,143.3,91.52,104.1,0.0
3,2004-06-01,1121.199951,16.30,188.900,6270.5,1.03,2.11,6.01,5.6,142.9,91.73,103.4,1.0
4,2004-07-01,1128.939941,15.20,189.100,6284.3,1.26,1.93,5.82,5.5,143.2,91.95,103.9,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,2020-08-03,3294.610107,24.28,259.580,18357.4,0.10,0.45,2.25,8.4,193.0,121.22,122.1,1.0
198,2020-09-01,3526.649902,26.12,260.190,18575.2,0.09,0.55,2.31,7.9,192.9,121.97,122.8,1.0
199,2020-10-01,3380.800049,26.70,260.352,18735.7,0.09,0.54,2.35,6.9,193.7,122.93,123.0,0.0
200,2020-11-02,3310.239990,37.13,260.721,18969.8,0.09,0.71,2.30,6.7,194.5,123.41,124.0,0.0


In [4]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [30]:
dataset = df.values
dataset[:,2:12]

array([[14.44, 187.1, 6150.1, ..., 140.3, 90.98, 103.0],
       [16.65, 187.4, 6191.2, ..., 141.8, 91.28, 103.7],
       [16.62, 188.2, 6268.1, ..., 143.3, 91.52, 104.1],
       ...,
       [26.7, 260.352, 18735.7, ..., 193.7, 122.93, 123.0],
       [37.13, 260.721, 18969.8, ..., 194.5, 123.41, 124.0],
       [20.77, 261.564, 19124.7, ..., 196.7, 123.66, 125.5]], dtype=object)

In [31]:
# split into input (X) and output (Y) variables
X = dataset[:,2:12].astype(float)
Y = dataset[:,-1]

In [32]:
Y

array([1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0,
       0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0,
       1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0,
       1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0,
       0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0,
       1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0,
       0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0,
       0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0,
       0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0,
       1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0,
       1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0,
       0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0,
       1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0,
       0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.

In [50]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [61]:
def create_model():
  model = Sequential()
  model.add(Dense(10, input_shape=(10,), activation='elu'))
  model.add(BatchNormalization())
  model.add(Dense(10, activation='elu'))
  model.add(BatchNormalization())
  model.add(Dense(8, activation='elu'))
  model.add(BatchNormalization())
  model.add(Dense(6, activation='elu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

  return model

In [62]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(model=create_model, epochs=100, batch_size=16, verbose=1)))
pipeline = Pipeline(estimators)

In [63]:
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)


Epoch 1/100
12/12 [==============================] - 2s 7ms/step - loss: 1.0256 - accuracy: 0.3702
Epoch 2/100
12/12 [==============================] - 0s 6ms/step - loss: 0.9238 - accuracy: 0.3978
Epoch 3/100
12/12 [==============================] - 0s 7ms/step - loss: 0.8170 - accuracy: 0.4199
Epoch 4/100
12/12 [==============================] - 0s 7ms/step - loss: 0.8212 - accuracy: 0.4530
Epoch 5/100
12/12 [==============================] - 0s 6ms/step - loss: 0.7545 - accuracy: 0.4641
Epoch 6/100
12/12 [==============================] - 0s 7ms/step - loss: 0.7571 - accuracy: 0.4696
Epoch 7/100
12/12 [==============================] - 0s 8ms/step - loss: 0.7149 - accuracy: 0.5304
Epoch 8/100
12/12 [==============================] - 0s 7ms/step - loss: 0.6787 - accuracy: 0.5856
Epoch 9/100
12/12 [==============================] - 0s 7ms/step - loss: 0.6979 - accuracy: 0.5691
Epoch 10/100
12/12 [==============================] - 0s 7ms/step - loss: 0.6860 - accuracy: 0.5801
Epoch 11/

In [54]:
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 69.90% (10.63%)
